In [13]:
from robotic import ry
import numpy as np
import time

###########################################PART-2 STARTS HERE##########################################

In [14]:
C = ry.Config()
C.addFile('cargobot.g')
C.addFile('maze.g')
C.addFile('cargo.g')

qHome = C.getJointState()
C.setJointState(qHome)

Get Final Pose with KOMO

In [18]:
S = ry.Skeleton()

S.addEntry([1., 1.], ry.SY.touch,  ["l_gripper", "cargo_handle"])

S.addEntry([1., 4.], ry.SY.stable,  ["l_gripper", "cargo_handle"])
S.addEntry([2., 2.], ry.SY.touch,  ["cargo", "cp1"])
S.addEntry([3., 3.], ry.SY.touch,  ["cargo", "cp2"])
S.addEntry([4., 4.], ry.SY.touch,  ["cargo", "final"])
S.enableAccumulatedCollisions(True)

komo = S.getKomo_waypoints(C, 1e-1, 1e-2,0)
nlp = komo.nlp()
sol = ry.NLP_Solver()
sol.setProblem(nlp)
sol.setOptions( stopTolerance=1e-2 )
ret = sol.solve()
waypoints = komo.getPath_qAll()
#waypoints = komo.getPath()[0]
komo.view_play(True, .02)
#C.setJointState(q[q.shape[0]-1])

====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
==nlp== it:   0  evals: 529  A(x):    10.5343  f:    0.24162  g:          0  h:    6.40453  |x-x'|:    3.48694 	stop:DeltaConverge
==nlp== it:   1  evals: 529  A(x):    72.2901  mu:5
==nlp== it:   1  evals: 608  A(x):    -1.8246  f:   0.226861  g:          0  h:    1.17437  |x-x'|:   0.153775 	stop:TinyXSteps (bad:1)
==nlp== it:   2  evals: 608  A(x):    9.57568  mu:25
==nlp== it:   2  evals: 996  A(x):   0.194442  f:   0.195056  g:          0  h: 0.00882144  |x-x'|:   0.340978 	stop:TinyXSteps (bad:2)
==nlp== it:   3  evals: 996  A(x):   0.197766  mu:125
==nlp== it:   3  evals:1001  A(x):   0.197766  f:   0.195056  g:          0  h: 0.00892458  |x-x'|:          0 	stop:CritEvals (bad:3)
==nlp== StoppingCriterion Delta<0.01


1

RRT

In [19]:
## solve for paths using RRT: for each phase create start-end problems, run RRT
m = len(waypoints)
rrt_dofs = []
rrt_paths = []
for t in range(0,int(m)):
    # grab config and waypoints
    [Ctmp, q0, q1] = S.getTwoWaypointProblem(t, komo)
    Ctmp.setJointState(q0);
    Ctmp.view(True, 'waypoint configuration phase ' + str(t) + ' START')
    Ctmp.setJointState(q1);
    Ctmp.view(True, 'waypoint configuration phase ' + str(t) + ' STOP')

    Ctmp.view(True, 'Continue')

    # call path finder
    sol = ry.PathFinder()
    sol.setProblem(Ctmp, q0, q1)
    ret = sol.solve()
    
    rrt_paths.append(ret.x)
    rrt_dofs.append(Ctmp.getDofIDs())

    #display the rrt path
    for i in range(0,ret.x.shape[0]):
        Ctmp.setJointState(ret.x[i])
        Ctmp.view(False, 'rrt path ' + str(i))
        time.sleep(.02)
"""
#define a path finding problem
m = len(waypoints)
rrt_dofs = []
rrt_paths = []
for i in range(0, int(m)):
    [Ctmp, q0, q1] = S.getTwoWaypointProblem(i, komo)
    Ctmp.setJointState(q0)
    Ctmp.setJointState(q1)
    
    rrt = ry.PathFinder()
    rrt.setProblem(Ctmp, q0, q1)
    ret = rrt.solve()
    path = ret.x
        
    if hasattr(ret, 'x') and isinstance(ret.x, np.ndarray) and ret.x.ndim == 2 and ret.x.shape[0] > 0:
        rrt_paths.append(ret.x)
        rrt_dofs.append(Ctmp.getDofIDs())
        for t in range(0, path.shape[0]):
            Ctmp.setJointState(path[t])
            Ctmp.view(False)
            time.sleep(0.02)
"""

IndexError: tuple index out of range

In [17]:
komo = S.getKomo_path(C, 60, 1e0, 1e0, 1e0,1e0)
komo.initWithWaypoints(waypoints)
komo.view(True, 'init with waypoints only')
for t in range(0,int(m)):
    komo.initPhaseWithDofsPath(t, rrt_dofs[t], rrt_paths[t], True)
    komo.view(True, 'init with RRT phase ' + str(t))

nlp = komo.nlp()
sol = ry.NLP_Solver()
sol.setProblem(nlp)
sol.setOptions( stopTolerance=1e-2 )
ret = sol.solve()
# report on result, view, and play
print(ret)
#print(nlp.report(2))
komo.view(True, 'path solution')
komo.view_play(True, .2)

====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
==nlp== it:   0  evals: 214  A(x):    1179.15  f:    1108.13  g:          0  h:    45.0952  |x-x'|:    2.57022 	stop:TinyXSteps (bad:1)
==nlp== it:   1  evals: 214  A(x):    1605.27  mu:5
==nlp== it:   1  evals: 226  A(x):    1605.27  f:    1108.13  g:          0  h:    45.2561  |x-x'|:          0 	stop:LineSearchSteps (bad:2)
==nlp== StoppingCriterion Delta<0.01
{ time: 11.1993, evals: 226, done: 1, feasible: 0, sos: 1108.13, f: 0, ineq: 0, eq: 45.2561 }


1